# Creating and running a fund
This script will import the vrious models and run them as funds from 01-01-2014 to find the best performing fund/model.

In [1]:
#Import models
import numpy as np
import pandas as pd
import math
import os
import tables
from rf_modules import *

In [2]:
#Setup variables
path = r'C:\Users\Robert\Documents\python_scripts\stock_trading_ml_modelling\historical_prices'

In [3]:
#Create a class for maintaining the value of the fund
class new_fund:
    def __init__(self,init_val):
        self.st_val = init_val #The starting value of the fund
        self.available = init_val #The amount available to invest, initially set to the investment value
        self.invested_value = 0 #The total value of hel stocks
        self.codb = 0 #The sum cost of doing business (stamp duty, trading fees)
        self.ledger = [] #A running total of the trades made in this fund
        self.cur_holdings = {} #A dictionary of the current fund holdings
        print('NEW FUND')
        print('st_val:{:,}'.format(self.st_val))
        print('available:{:,}'.format(self.available))
    @property
    def fund_value(self):
        return self.invested_value + self.available
    #Create a function to buy shares
    def buy(self
            ,ticker:str
            ,trade_date:int
            ,price:float
            ,spread:float
            ,value:float
            ,trade_cost:float
            ,signal_prob:float
            ,val_inc_tc:bool = True
            ,_verbose:bool = True):
        #Check if already bought
        if ticker in self.cur_holdings:
            return
        #Error check
        if spread > 1 or spread < 0:
            raise ValueError('spread should be between 0 and 1, the value expressed was -> {:,.2f}'.format(spread))
        if price < 0:
            raise ValueError('price cannot be a negative, the value expressed was -> {:,.2f}'.format(price))
        #Calculate the trade value
        if val_inc_tc == True: #Does the value include the trade cost
            trade_funds = value - trade_cost
        else:
            trade_funds = value
        #Calculate the ask and bid price of each share
        a_price = round(price * (1+spread),2)
        b_price = round(price * (1-spread),2)
        #Calculate the number of whole shares which can be purchased
        share_vol = int(trade_funds/a_price)
        trade_value = round(share_vol * a_price,2)
        #Calc the ledger_value
        spread_cost = -round(share_vol * price * spread,2)
        ledger_value = -(trade_value + trade_cost)
        #Check the fund has the money to cover this trade
        if ledger_value > self.available:
            raise ValueError('you do not have the funds to make this trade -> this transaction will be cancelled')
        #Create a record for the ledger
        ledge_rec = {
            'trade_type':'buy'
            ,'signal_prob':signal_prob
            ,'ticker':ticker
            ,'trade_date':trade_date
            ,'spread':spread
            ,'price':price
            ,'ask_price':a_price
            ,'bid_price':b_price
            ,'share_vol':share_vol
            ,'trade_value':trade_value
            ,'trade_cost':trade_cost
            ,'spread_cost':spread_cost
            ,'holding_value':share_vol*price
            ,'ledger_value':ledger_value
            ,'invested_pre_trade':self.invested_value
            ,'invested_post_trade':self.invested_value + (share_vol*price)
            ,'available_pre_trade':self.available
            ,'available_post_trade':self.available + ledger_value
        }
        self.ledger.append(ledge_rec)
        if _verbose == True:
            print('LEDGER ENTRY -> \n\t{}'.format(ledge_rec)) #OPTIONAL - PRINT ENTRY
        #Update the object
        self.available += round(ledger_value,2)
        self.codb += round(trade_cost + spread_cost,2)
        self.invested_value += round(share_vol*price,2)
        #Add to cur_holdings
        holding_rec = {
           'share_vol':share_vol
            ,'cur_price':price
            ,'value':round(share_vol*price,2)
        }
        #Check if key already in dict
        if ticker in self.cur_holdings: #Should not happen due to earlier exit if key in cur_holdings
            self.cur_holdings[ticker]['share_vol'] += share_vol
            self.cur_holdings[ticker]['cur_price'] = round(price,2)
            self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
        else:
            self.cur_holdings[ticker] = holding_rec
        
    #Create a function to sell shares
    def sell(self,ticker:str,trade_date:int,price:float,spread:float,trade_cost:float,signal_prob:float):
        #Check if already sold
        if ticker not in self.cur_holdings:
            return
        #Round down the share_vol
        share_vol = int(self.cur_holdings[ticker]['share_vol'])
        #Error check
        if spread > 1 or spread < 0:
            raise ValueError('spread should be between 0 and 1, the value expressed was -> {}'.format(spread))
        if price < 0:
            raise ValueError('price cannot be a negative, the value expressed was -> {}'.format(price))
        #Calculate the ask and bid price of each share
        a_price = round(price * (1+spread),2)
        b_price = round(price * (1-spread),2)
        #Calculate the trade value
        trade_value = round(share_vol*b_price,2)
        value = round(trade_value - trade_cost,2)
        spread_cost = round(share_vol * price * spread)
        #Calc the ledger_value
        ledger_value = round(trade_value - trade_cost,2)
        #Create a record for the ledger
        ledge_rec = {
           'trade_type':'sell'
            ,'signal_prob':signal_prob
            ,'ticker':ticker
            ,'trade_date':trade_date
            ,'spread':spread
            ,'price':price
            ,'ask_price':a_price
            ,'bid_price':b_price
            ,'share_vol':share_vol
            ,'trade_value':trade_value
            ,'trade_cost':trade_cost
            ,'spread_cost':spread_cost
            ,'holding_value':share_vol*price
            ,'ledger_value':ledger_value
            ,'invested_pre_trade':self.invested_value
            ,'invested_post_trade':self.invested_value - (share_vol*price)
            ,'available_pre_trade':self.available
            ,'available_post_trade':self.available + ledger_value
        }
        self.ledger.append(ledge_rec)
        print('LEDGER ENTRY -> \n\t{}'.format(ledge_rec)) #OPTIONAL - PRINT ENTRY
        #Update the object
        self.available += round(ledger_value,2)
        self.codb += round(trade_cost + spread_cost,2)
        self.invested_value += -round((share_vol*price),2)
        #Remove from cur_holdings
        #Check if key already in dict
        if ticker in self.cur_holdings:
            if self.cur_holdings[ticker]['share_vol'] > share_vol:
                self.cur_holdings[ticker]['share_vol'] += -share_vol
                self.cur_holdings[ticker]['cur_price'] = price
                self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
            elif self.cur_holdings[ticker]['share_vol'] == share_vol:
                del self.cur_holdings[ticker] #Delete from the dictionary
            else:
                raise ValueError('you do not have enough shares to make this trade. You want to sell {} of {} however you only have {}'.format(share_vol,ticker,self.cur_holdings[ticker]['share_vol']))
        else:
            return
        
    #Create a function to update value after a price change
    def price_change(self,ticker:str,price:float):
        #Check if key already in dict
        if ticker in self.cur_holdings:
            #Update the object
            self.invested_value += round((self.cur_holdings[ticker]['share_vol']*price) - self.cur_holdings[ticker]['value'],2)
            self.cur_holdings[ticker]['cur_price'] = price
            self.cur_holdings[ticker]['value'] = round(self.cur_holdings[ticker]['share_vol']*price,2)
        else:
            return

In [4]:
def completed_trades(_df_in,_init_val:int=1,_cur_holdings:dict={}):
    #From the ledger create a dataframe of completed trades
    _completed_trades = {}
    #Format
    #     ABC:{
    #         open_position:True/False #Bool showing if there is currently an open position
    #         ,trades:[ #List showing all trades
    #             { #Each trade has an object
    #                 share_vol:12345 #Volume of shares purchased
    #                 ,buy_spend:12345.67 #Total value spent in buying shares including costs
    #                 ,sell_spend:12345.67 #Total value spent in selling shares including costs
    #                 ,profit_loss:12345.67 #Profit/loss of ths trade
    #                 ,holding_value:12345.67 #Current value of the shares
    #             }
    #         ]
    #     }
    for _index,_row in _df_in.iterrows():
        #Check if there is an open trade for this ticker
        if _row['ticker'] not in _completed_trades:
            _completed_trades[_row['ticker']] = {
                'open_position':False
                ,'trades':[]
            }
        #Deal with buying
        if _row['trade_type'] == 'buy':
            #Create a trade object and add to the trades list in completed_trades
            _completed_trades[_row['ticker']]['trades'].append({
                'ticker':_row['ticker']
                ,'share_vol':_row['share_vol']
                ,'buy_spend':-_row['ledger_value']
                ,'buy_prob':_row['signal_prob']
                ,'buy_date':_row['trade_date']
                ,'sell_spend':None
                ,'sell_prob':None
                ,'sell_date':None
                ,'periods_held':None
                ,'profit_loss':None
                ,'holding_value':_row['holding_value']
            })
            #open the trading position
            _completed_trades[_row['ticker']]['open_position'] = True
        #Dealing with selling
        elif _row['trade_type'] == 'sell':
            _shares_to_sell = _row['share_vol']
            #Find open positions and sell until shares al gone
            for _trade in _completed_trades[_row['ticker']]['trades']:
                if _trade['share_vol'] == _row['share_vol']:
                    _trade['sell_spend'] = _row['ledger_value']
                    _trade['sell_prob'] = _row['signal_prob']
                    _trade['sell_date'] = _row['trade_date']
                    _trade['holding_value'] = _row['holding_value']
    _trades_li = []
    for _tick in _completed_trades:
        for _trade in _completed_trades[_tick]['trades']:
            _trades_li.append(_trade)
    _trades_df = pd.DataFrame(_trades_li,columns=[
                'ticker'
                ,'share_vol'
                ,'buy_spend'
                ,'buy_prob'
                ,'buy_date'
                ,'sell_spend'
                ,'sell_prob'
                ,'sell_date'
                ,'periods_held'
                ,'profit_loss'
                ,'holding_value'])
    _trades_df['profit_loss'] = _trades_df['sell_spend'] - _trades_df['buy_spend'] 
    _trades_df['periods_held'] = _trades_df['sell_date'] - _trades_df['buy_date']
    _trades_df['periods_held'] = [_x.days/7 for _x in _trades_df['periods_held']]
    #Dealing with open trade values
    _trades_df.reset_index(inplace=True,drop=True)
    for _index,_row in _trades_df[_trades_df.sell_date.isna()].iterrows():
        _trades_df.iloc[_index,_trades_df.columns.get_loc('holding_value')] = _cur_holdings[_row.ticker]['value']
    #Display top level summary
    print('TRADE COUNT -> {:,}'.format(_trades_df[~_trades_df['profit_loss'].isna()].shape[0]))
    print('TOTAL PROFIT/LOSS -> £{:,.2f}'.format(_trades_df['profit_loss'].sum()/100))
    print('\nOPEN TRADES -> {:,}'.format(_trades_df[_trades_df['profit_loss'].isna()].shape[0]))
    print('OPEN TRADES VALUE -> £{:,.2f}'.format(_trades_df[_trades_df['profit_loss'].isna()]['holding_value'].sum()/100))
    print('\nTOTAL ROI VALUE -> £{:,.2f}'.format((_trades_df['profit_loss'].sum()/100) + (_trades_df[_trades_df['profit_loss'].isna()]['holding_value'].sum()/100)))
    print('TOTAL ROI % -> {:,.2f}%'.format(100 * (_trades_df['profit_loss'].sum() + _trades_df[_trades_df['profit_loss'].isna()]['holding_value'].sum() - _init_val) / _init_val))
    return _trades_df

In [5]:
def run_fund(_df_in,_fund_value_st:float=1000000,_investment_limit_min_val:float=100000,_investment_limit_max_per:float=0.1,_spread:float=0.01,_trade_cost:float=250):
    #Run through rows and buy and sell according to signals and holdings
    _run_time = process_time()
    _fund = new_fund(_fund_value_st)
    try:
        for _index,_row in _df_in.iterrows():
            #Follow signal
            if _row['signal'] =='buy':
                #Check for funds
                if _fund.available < _investment_limit_min_val:
                    continue
                #Buy shares
                _val_to_invest = _fund.available*_investment_limit_max_per if _fund.available*_investment_limit_max_per > _investment_limit_min_val else _investment_limit_min_val
                _fund.buy(_row['ticker'],_row['date'],_row['open_shift_neg1'],_spread,_val_to_invest,_trade_cost,_row['signal_prob'])
            elif _row['signal'] =='sell':
                #Sell shares
                _fund.sell(_row['ticker'],_row['date'],_row['open_shift_neg1'],_spread,_trade_cost,_row['signal_prob'])
            elif _row['signal'] == 'hold':
                _fund.price_change(_row['ticker'],_row['close'])
        _run_time.end()
    except Exception as e:
        print('ERROR: '.format(e))
    return _fund

In [6]:
#Establish trading variables
#All figures in pence
fund_vars = {
    '_fund_value_st':1000000 #£10,000
    ,'_trade_cost':250 #£2.50
    ,'_investment_limit_min_val':100000 #£1,000
    ,'_investment_limit_max_per':0.1 #10%
    ,'_spread':0.01 #1%
}

# Light Gradient Boost
Steps:
1. Import the output from running the model
2. Run the model through the fund

In [7]:
#Import and combine prices files
df_prices = pd.read_hdf(path +r'\historic_lgb_bsh_signals.h5')
print('SHAPE: {}'.format(df_prices.shape))
print(df_prices.columns)
df_prices.head()

SHAPE: (81602, 20)
Index(['ticker', 'date', 'high', 'low', 'volume', 'open', 'close', 'ema12',
       'ema26', 'macd_line', 'signal_line', 'macd', 'change_price',
       'per_change_price', 'buy', 'sell', 'hold', 'open_shift_neg1', 'signal',
       'signal_prob'],
      dtype='object')


,ticker,date,high,low,volume,open,close,ema12,ema26,macd_line,signal_line,macd,change_price,per_change_price,buy,sell,hold,open_shift_neg1,signal,signal_prob
237,3IN,2014-01-06,190.56,187.61,2305949.0,189.72,189.72,187.437790,187.351381,0.086408,-0.035103,0.121512,0.00,0.000000,False,False,True,188.87,hold,0.719285
238,3IN,2014-01-13,190.56,188.30,2091342.0,188.87,189.58,187.767361,187.516464,0.250896,0.022097,0.228800,0.71,0.003759,False,False,True,188.87,hold,0.712502
239,3IN,2014-01-20,190.00,187.47,2386674.0,188.87,189.58,188.046228,187.669319,0.376910,0.093059,0.283850,0.71,0.003759,False,False,True,188.17,hold,0.735956
240,3IN,2014-01-27,190.84,188.17,1783577.0,188.17,190.84,188.476039,187.904184,0.571855,0.188819,0.383037,2.67,0.014189,False,False,True,189.44,sell,0.615498
241,3IN,2014-02-03,190.42,188.31,1742104.0,189.44,189.72,188.667418,188.038689,0.628729,0.276801,0.351928,0.28,0.001478,False,False,True,188.59,hold,0.621555


In [8]:
#Order the data by date (asc) and buy probability (desc)
signal_df = df_prices[['ticker','date','open','close','open_shift_neg1','signal','signal_prob']].copy()
signal_df.sort_values(['date','signal_prob'],ascending=[True,False],inplace=True)
signal_df.reset_index(drop=True,inplace=True)
print(signal_df.signal.value_counts())
signal_df.head()

hold    58652
sell    14484
buy      8466
Name: signal, dtype: int64


,ticker,date,open,close,open_shift_neg1,signal,signal_prob
0,ASHM,2014-01-06,399.8,407.90,414.00,sell,0.910613
1,FRES,2014-01-06,772.0,679.00,690.00,buy,0.904725
2,BGEO,2014-01-06,2449.0,2500.00,2513.00,sell,0.877215
3,BHP,2014-01-06,1725.4,1660.58,1671.77,buy,0.846647
4,AAL,2014-01-06,1290.5,1249.50,1256.50,buy,0.846176


In [9]:
#Run through rows and buy and sell according to signals and holdings
lgb_fund = run_fund(signal_df,**fund_vars)

NEW FUND
st_val:1,000,000
available:1,000,000
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.9047253980922231, 'ticker': 'FRES', 'trade_date': Timestamp('2014-01-06 00:00:00'), 'spread': 0.01, 'price': 690.0, 'ask_price': 696.9, 'bid_price': 683.1, 'share_vol': 143, 'trade_value': 99656.7, 'trade_cost': 250, 'spread_cost': -986.7, 'holding_value': 98670.0, 'ledger_value': -99906.7, 'invested_pre_trade': 0, 'invested_post_trade': 98670.0, 'available_pre_trade': 1000000, 'available_post_trade': 900093.3}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.8466466285657037, 'ticker': 'BHP', 'trade_date': Timestamp('2014-01-06 00:00:00'), 'spread': 0.01, 'price': 1671.77, 'ask_price': 1688.49, 'bid_price': 1655.05, 'share_vol': 59, 'trade_value': 99620.91, 'trade_cost': 250, 'spread_cost': -986.34, 'holding_value': 98634.43, 'ledger_value': -99870.91, 'invested_pre_trade': 98670.0, 'invested_post_trade': 197304.43, 'available_pre_trade': 900093.3, 'available_post_trade': 800

LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.6954388388499855, 'ticker': 'WTB', 'trade_date': Timestamp('2014-04-07 00:00:00'), 'spread': 0.01, 'price': 4005.0, 'ask_price': 4045.05, 'bid_price': 3964.95, 'share_vol': 24, 'trade_value': 97081.2, 'trade_cost': 250, 'spread_cost': -961.2, 'holding_value': 96120.0, 'ledger_value': -97331.2, 'invested_pre_trade': 720509.53, 'invested_post_trade': 816629.53, 'available_pre_trade': 476577.13999999996, 'available_post_trade': 379245.93999999994}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.6791813695918445, 'ticker': 'ABF', 'trade_date': Timestamp('2014-04-07 00:00:00'), 'spread': 0.01, 'price': 2640.0, 'ask_price': 2666.4, 'bid_price': 2613.6, 'share_vol': 37, 'trade_value': 98656.8, 'trade_cost': 250, 'spread_cost': -976.8, 'holding_value': 97680.0, 'ledger_value': -98906.8, 'invested_pre_trade': 816629.53, 'invested_post_trade': 914309.53, 'available_pre_trade': 379245.93999999994, 'available_post_trade': 280339.139

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.8809753383321528, 'ticker': 'IAG', 'trade_date': Timestamp('2014-06-02 00:00:00'), 'spread': 0.01, 'price': 420.7, 'ask_price': 424.91, 'bid_price': 416.49, 'share_vol': 249, 'trade_value': 103706.01, 'trade_cost': 250, 'spread_cost': 1048, 'holding_value': 104754.3, 'ledger_value': 103456.01, 'invested_pre_trade': 1245071.0700000005, 'invested_post_trade': 1140316.7700000005, 'available_pre_trade': 43530.65000000002, 'available_post_trade': 146986.66000000003}
LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.8789480984331656, 'ticker': 'SMP', 'trade_date': Timestamp('2014-06-02 00:00:00'), 'spread': 0.01, 'price': 389.5, 'ask_price': 393.39, 'bid_price': 385.61, 'share_vol': 265, 'trade_value': 102186.65, 'trade_cost': 250, 'spread_cost': 1032, 'holding_value': 103217.5, 'ledger_value': 101936.65, 'invested_pre_trade': 1140316.7700000005, 'invested_post_trade': 1037099.2700000005, 'available_pre_trade': 146986.66000000

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.8029259306191048, 'ticker': 'SXX', 'trade_date': Timestamp('2014-06-30 00:00:00'), 'spread': 0.01, 'price': 12.424, 'ask_price': 12.55, 'bid_price': 12.3, 'share_vol': 8453, 'trade_value': 103971.9, 'trade_cost': 250, 'spread_cost': 1050, 'holding_value': 105020.072, 'ledger_value': 103721.9, 'invested_pre_trade': 1333077.98, 'invested_post_trade': 1228057.908, 'available_pre_trade': 5251.06999999992, 'available_post_trade': 108972.96999999991}
LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.8018688301880825, 'ticker': 'GFTU', 'trade_date': Timestamp('2014-06-30 00:00:00'), 'spread': 0.01, 'price': 601.0, 'ask_price': 607.01, 'bid_price': 594.99, 'share_vol': 180, 'trade_value': 107098.2, 'trade_cost': 250, 'spread_cost': 1082, 'holding_value': 108180.0, 'ledger_value': 106848.2, 'invested_pre_trade': 1228057.91, 'invested_post_trade': 1119877.91, 'available_pre_trade': 108972.96999999991, 'available_post_trade': 21582

LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.5321217549490829, 'ticker': 'PTEC', 'trade_date': Timestamp('2014-07-21 00:00:00'), 'spread': 0.01, 'price': 608.0, 'ask_price': 614.08, 'bid_price': 601.92, 'share_vol': 162, 'trade_value': 99480.96, 'trade_cost': 250, 'spread_cost': -984.96, 'holding_value': 98496.0, 'ledger_value': -99730.96, 'invested_pre_trade': 1165422.6199999999, 'invested_post_trade': 1263918.6199999999, 'available_pre_trade': 215329.0899999998, 'available_post_trade': 115598.12999999979}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.5132429831236978, 'ticker': 'RMV', 'trade_date': Timestamp('2014-07-21 00:00:00'), 'spread': 0.01, 'price': 217.6, 'ask_price': 219.78, 'bid_price': 215.42, 'share_vol': 453, 'trade_value': 99560.34, 'trade_cost': 250, 'spread_cost': -985.73, 'holding_value': 98572.8, 'ledger_value': -99810.34, 'invested_pre_trade': 1263918.6199999999, 'invested_post_trade': 1362491.42, 'available_pre_trade': 115598.12999999979, 'a

...

	{'trade_type': 'buy', 'signal_prob': 0.566465543147705, 'ticker': 'SPT', 'trade_date': Timestamp('2019-05-06 00:00:00'), 'spread': 0.01, 'price': 154.2, 'ask_price': 155.74, 'bid_price': 152.66, 'share_vol': 2773, 'trade_value': 431867.02, 'trade_cost': 250, 'spread_cost': -4275.97, 'holding_value': 427596.6, 'ledger_value': -432117.02, 'invested_pre_trade': 63875813.550000094, 'invested_post_trade': 64303410.150000095, 'available_pre_trade': 4321953.02999999, 'available_post_trade': 3889836.00999999}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.5521009953317544, 'ticker': 'BGSC', 'trade_date': Timestamp('2019-05-06 00:00:00'), 'spread': 0.01, 'price': 1328.0, 'ask_price': 1341.28, 'bid_price': 1314.72, 'share_vol': 289, 'trade_value': 387629.92, 'trade_cost': 250, 'spread_cost': -3837.92, 'holding_value': 383792.0, 'ledger_value': -387879.92, 'invested_pre_trade': 64281937.150000095, 'invested_post_trade': 64665729.150000095, 'available_pre_trade': 3889836.00999999, 'avai

	{'trade_type': 'buy', 'signal_prob': 0.5928381833644393, 'ticker': 'GSS', 'trade_date': Timestamp('2019-06-10 00:00:00'), 'spread': 0.01, 'price': 734.0, 'ask_price': 741.34, 'bid_price': 726.66, 'share_vol': 175, 'trade_value': 129734.5, 'trade_cost': 250, 'spread_cost': -1284.5, 'holding_value': 128450.0, 'ledger_value': -129984.5, 'invested_pre_trade': 73311936.33, 'invested_post_trade': 73440386.33, 'available_pre_trade': 1302003.1699999804, 'available_post_trade': 1172018.6699999804}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.5897357973558903, 'ticker': 'CARD', 'trade_date': Timestamp('2019-06-10 00:00:00'), 'spread': 0.01, 'price': 175.1, 'ask_price': 176.85, 'bid_price': 173.35, 'share_vol': 661, 'trade_value': 116897.85, 'trade_cost': 250, 'spread_cost': -1157.41, 'holding_value': 115741.09999999999, 'ledger_value': -117147.85, 'invested_pre_trade': 73438679.33, 'invested_post_trade': 73554420.42999999, 'available_pre_trade': 1172018.6699999804, 'available_post_t

	{'trade_type': 'buy', 'signal_prob': 0.7215298919702777, 'ticker': 'RDW', 'trade_date': Timestamp('2019-07-29 00:00:00'), 'spread': 0.01, 'price': 542.0, 'ask_price': 547.42, 'bid_price': 536.58, 'share_vol': 4694, 'trade_value': 2569589.48, 'trade_cost': 250, 'spread_cost': -25441.48, 'holding_value': 2544148.0, 'ledger_value': -2569839.48, 'invested_pre_trade': 55292503.75000007, 'invested_post_trade': 57836651.75000007, 'available_pre_trade': 25701511.089999985, 'available_post_trade': 23131671.609999985}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.71629113417242, 'ticker': 'SNR', 'trade_date': Timestamp('2019-07-29 00:00:00'), 'spread': 0.01, 'price': 201.0, 'ask_price': 203.01, 'bid_price': 198.99, 'share_vol': 11393, 'trade_value': 2312892.93, 'trade_cost': 250, 'spread_cost': -22899.93, 'holding_value': 2289993.0, 'ledger_value': -2313142.93, 'invested_pre_trade': 57836651.75000007, 'invested_post_trade': 60126644.75000007, 'available_pre_trade': 23131671.609999985

	{'trade_type': 'sell', 'signal_prob': 0.8959842559808671, 'ticker': 'ISAT', 'trade_date': Timestamp('2019-08-26 00:00:00'), 'spread': 0.01, 'price': 595.0, 'ask_price': 600.95, 'bid_price': 589.05, 'share_vol': 6526, 'trade_value': 3844140.3, 'trade_cost': 250, 'spread_cost': 38830, 'holding_value': 3882970.0, 'ledger_value': 3843890.3, 'invested_pre_trade': 85634867.68, 'invested_post_trade': 81751897.68, 'available_pre_trade': 1278474.5399999775, 'available_post_trade': 5122364.8399999775}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.8653739223015193, 'ticker': 'BNZL', 'trade_date': Timestamp('2019-08-26 00:00:00'), 'spread': 0.01, 'price': 2010.0, 'ask_price': 2030.1, 'bid_price': 1989.9, 'share_vol': 252, 'trade_value': 511585.2, 'trade_cost': 250, 'spread_cost': -5065.2, 'holding_value': 506520.0, 'ledger_value': -511835.2, 'invested_pre_trade': 81751897.68, 'invested_post_trade': 82258417.68, 'available_pre_trade': 5122364.8399999775, 'available_post_trade': 4610529.

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.5816689484444132, 'ticker': 'KAZ', 'trade_date': Timestamp('2019-09-16 00:00:00'), 'spread': 0.01, 'price': 442.8, 'ask_price': 447.23, 'bid_price': 438.37, 'share_vol': 498, 'trade_value': 218308.26, 'trade_cost': 250, 'spread_cost': 2205, 'holding_value': 220514.4, 'ledger_value': 218058.26, 'invested_pre_trade': 87275723.48000008, 'invested_post_trade': 87055209.08000007, 'available_pre_trade': 8889901.729999974, 'available_post_trade': 9107959.989999974}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.5775555571479922, 'ticker': 'JII', 'trade_date': Timestamp('2019-09-16 00:00:00'), 'spread': 0.01, 'price': 730.0, 'ask_price': 737.3, 'bid_price': 722.7, 'share_vol': 1234, 'trade_value': 909828.2, 'trade_cost': 250, 'spread_cost': -9008.2, 'holding_value': 900820.0, 'ledger_value': -910078.2, 'invested_pre_trade': 87065079.47000007, 'invested_post_trade': 87965899.47000007, 'available_pre_trade': 9107959.989999974, '

In [10]:
#Show summary
print('lgb_fund.st_val:£{:,.2f}'.format(lgb_fund.st_val/100))
print('lgb_fund.available:£{:,.2f}'.format(lgb_fund.available/100))
print('lgb_fund.codb:£{:,.2f}'.format(lgb_fund.codb/100))
print('lgb_fund.invested_value:£{:,.2f}'.format(lgb_fund.invested_value/100))
lgb_ledger_df = pd.DataFrame(lgb_fund.ledger,columns=[
            'trade_type'
            ,'signal_prob'
            ,'ticker'
            ,'trade_date'
            ,'spread'
            ,'price'
            ,'ask_price'
            ,'bid_price'
            ,'share_vol'
            ,'trade_value'
            ,'trade_cost'
            ,'spread_cost'
            ,'holding_value'
            ,'ledger_value'
            ,'invested_pre_trade'
            ,'invested_post_trade'
            ,'available_pre_trade'
            ,'available_post_trade'])
print('TRADE COUNT:{:,}'.format(len(lgb_ledger_df)))
lgb_ledger_df.head(50)

lgb_fund.st_val:£10,000.00
lgb_fund.available:£74,222.11
lgb_fund.codb:£20,740.03
lgb_fund.invested_value:£879,824.93
TRADE COUNT:6,906


,trade_type,signal_prob,ticker,trade_date,spread,price,ask_price,bid_price,share_vol,trade_value,trade_cost,spread_cost,holding_value,ledger_value,invested_pre_trade,invested_post_trade,available_pre_trade,available_post_trade
0,buy,0.904725,FRES,2014-01-06,0.01,690.00,696.90,683.10,143,99656.70,250,-986.70,98670.00,-99906.70,0.00,98670.00,1000000.00,900093.30
1,buy,0.846647,BHP,2014-01-06,0.01,1671.77,1688.49,1655.05,59,99620.91,250,-986.34,98634.43,-99870.91,98670.00,197304.43,900093.30,800222.39
2,buy,0.846176,AAL,2014-01-06,0.01,1256.50,1269.07,1243.93,78,98987.46,250,-980.07,98007.00,-99237.46,197304.43,295311.43,800222.39,700984.93
3,buy,0.733076,GNC,2014-01-06,0.01,185.86,187.72,184.00,531,99679.32,250,-986.92,98691.66,-99929.32,295311.43,394003.09,700984.93,601055.61
4,buy,0.569527,VOD,2014-01-06,0.01,244.45,246.89,242.01,404,99743.56,250,-987.58,98757.80,-99993.56,394003.09,492760.89,601055.61,501062.05
5,buy,0.561968,BRBY,2014-01-06,0.01,1477.00,1491.77,1462.23,66,98456.82,250,-974.82,97482.00,-98706.82,492760.89,590242.89,501062.05,402355.23
6,buy,0.532325,ANTO,2014-01-06,0.01,785.00,792.85,777.15,125,99106.25,250,-981.25,98125.00,-99356.25,590242.89,688367.89,402355.23,302998.98
7,buy,0.519396,EVR,2014-01-06,0.01,102.70,103.73,101.67,961,99684.53,250,-986.95,98694.70,-99934.53,688367.89,787062.59,302998.98,203064.45
8,sell,0.740027,VOD,2014-01-13,0.01,251.07,253.58,248.56,404,100418.24,250,1014.00,101432.28,100168.24,787062.59,685630.31,203064.45,303232.69
9,sell,0.712033,BRBY,2014-01-13,0.01,1499.00,1513.99,1484.01,66,97944.66,250,989.00,98934.00,97694.66,685630.31,586696.31,303232.69,400927.35


In [11]:
#Get the completed trades
lgb_trades_df = completed_trades(lgb_ledger_df,lgb_fund.st_val,lgb_fund.cur_holdings)
lgb_trades_df

TRADE COUNT -> 3,364
TOTAL PROFIT/LOSS -> £1,240,226.97

OPEN TRADES -> 178
OPEN TRADES VALUE -> £1,314,572.79

TOTAL ROI VALUE -> £2,554,799.76
TOTAL ROI % -> 25,448.00%


,ticker,share_vol,buy_spend,buy_prob,buy_date,sell_spend,sell_prob,sell_date,periods_held,profit_loss,holding_value
0,FRES,143,99906.70,0.904725,2014-01-06,139549.66,0.736663,2014-02-17,6.0,39642.96,141212.50
1,FRES,127,99595.75,0.738016,2014-06-02,127365.95,0.833947,2014-08-04,9.0,27770.20,128905.00
2,FRES,178,184704.28,0.779585,2016-05-23,345141.20,0.721255,2016-08-08,11.0,160436.92,348880.00
3,FRES,60,99209.80,0.648871,2016-09-12,107501.60,0.823410,2016-09-26,2.0,8291.80,108840.00
4,FRES,88,98906.80,0.731763,2016-12-12,136876.88,0.837914,2017-02-06,8.0,37970.08,138512.00
5,FRES,119,166112.20,0.791881,2017-03-06,192840.59,0.759703,2017-04-10,5.0,26728.39,195041.00
6,FRES,126,179941.12,0.726081,2017-05-01,213928.58,0.951355,2017-06-05,5.0,33987.46,216342.00
7,FRES,69,98861.35,0.632543,2017-07-03,104537.54,0.619274,2017-07-24,3.0,5676.19,105846.00
8,FRES,431,638414.46,0.544307,2017-07-31,682027.31,0.869000,2017-09-04,5.0,43612.85,689169.00
9,FRES,727,948926.84,0.677536,2017-10-30,981461.72,0.899249,2017-11-20,3.0,32534.88,991628.00


In [12]:
#Export
lgb_ledger_df.to_csv(path+r'\trades_ledger_lgb_mod.csv')

# Random forest
Steps:
1. Import the output from runningthe model
2. Run the model through the fund

In [13]:
#Import and combine prices files
df_prices = pd.read_hdf(path +r'\historic_rf_bsh_signals.h5')
print('SHAPE: {}'.format(df_prices.shape))
print(df_prices.columns)
df_prices.head()

SHAPE: (88362, 7)
Index(['ticker', 'date', 'open', 'close', 'open_shift_neg1', 'signal',
       'signal_prob'],
      dtype='object')


,ticker,date,open,close,open_shift_neg1,signal,signal_prob
201,3IN,2014-01-06,189.72,189.72,188.87,sell,0.624721
202,3IN,2014-01-13,188.87,189.58,188.87,sell,0.624721
203,3IN,2014-01-20,188.87,189.58,188.17,sell,0.624721
204,3IN,2014-01-27,188.17,190.84,189.44,sell,0.615773
205,3IN,2014-02-03,189.44,189.72,188.59,sell,0.655506


In [14]:
#Order the data by date (asc) and buy probability (desc)
signal_df = df_prices[['ticker','date','open','close','open_shift_neg1','signal','signal_prob']].copy()
signal_df.sort_values(['date','signal_prob'],ascending=[True,False],inplace=True)
signal_df.reset_index(drop=True,inplace=True)
signal_df.head()

,ticker,date,open,close,open_shift_neg1,signal,signal_prob
0,RR,2014-01-06,1272.00,1239.00,1239.00,sell,0.669891
1,IMI,2014-01-06,1742.86,1786.29,1789.71,sell,0.662097
2,NBLS,2014-01-06,105.30,105.90,105.80,sell,0.656348
3,BAB,2014-01-06,1210.58,1226.45,1224.68,sell,0.640044
4,TEP,2014-01-06,1753.00,1814.00,1815.00,sell,0.637062


In [15]:
#Run through rows and buy and sell according to signals and holdings
rf_fund = run_fund(signal_df,**fund_vars)

NEW FUND
st_val:1,000,000
available:1,000,000
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3633358681469089, 'ticker': 'AGR', 'trade_date': Timestamp('2014-01-06 00:00:00'), 'spread': 0.01, 'price': 40.0, 'ask_price': 40.4, 'bid_price': 39.6, 'share_vol': 2469, 'trade_value': 99747.6, 'trade_cost': 250, 'spread_cost': -987.6, 'holding_value': 98760.0, 'ledger_value': -99997.6, 'invested_pre_trade': 0, 'invested_post_trade': 98760.0, 'available_pre_trade': 1000000, 'available_post_trade': 900002.4}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3633358681469089, 'ticker': 'CNE', 'trade_date': Timestamp('2014-01-06 00:00:00'), 'spread': 0.01, 'price': 271.4, 'ask_price': 274.11, 'bid_price': 268.69, 'share_vol': 363, 'trade_value': 99501.93, 'trade_cost': 250, 'spread_cost': -985.18, 'holding_value': 98518.2, 'ledger_value': -99751.93, 'invested_pre_trade': 98760.0, 'invested_post_trade': 197278.2, 'available_pre_trade': 900002.4, 'available_post_trade': 800250.47}


LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.36031595810221667, 'ticker': 'SGRO', 'trade_date': Timestamp('2015-02-09 00:00:00'), 'spread': 0.01, 'price': 405.93, 'ask_price': 409.99, 'bid_price': 401.87, 'share_vol': 303, 'trade_value': 121766.61, 'trade_cost': 250, 'spread_cost': 1230, 'holding_value': 122996.79000000001, 'ledger_value': 121516.61, 'invested_pre_trade': 919979.7500000001, 'invested_post_trade': 796982.9600000001, 'available_pre_trade': 57103.42999999995, 'available_post_trade': 178620.03999999995}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3591986118770581, 'ticker': 'KAZ', 'trade_date': Timestamp('2015-02-09 00:00:00'), 'spread': 0.01, 'price': 255.0, 'ask_price': 257.55, 'bid_price': 252.45, 'share_vol': 387, 'trade_value': 99671.85, 'trade_cost': 250, 'spread_cost': -986.85, 'holding_value': 98685.0, 'ledger_value': -99921.85, 'invested_pre_trade': 796982.9600000001, 'invested_post_trade': 895667.9600000001, 'available_pre_trade': 178620

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3496861844539072, 'ticker': 'KAZ', 'trade_date': Timestamp('2016-11-07 00:00:00'), 'spread': 0.01, 'price': 364.1, 'ask_price': 367.74, 'bid_price': 360.46, 'share_vol': 387, 'trade_value': 139498.02, 'trade_cost': 250, 'spread_cost': 1409, 'holding_value': 140906.7, 'ledger_value': 139248.02, 'invested_pre_trade': 857565.4600000001, 'invested_post_trade': 716658.76, 'available_pre_trade': 97059.04, 'available_post_trade': 236307.06}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3491206697941979, 'ticker': 'BME', 'trade_date': Timestamp('2016-11-07 00:00:00'), 'spread': 0.01, 'price': 237.5, 'ask_price': 239.88, 'bid_price': 235.12, 'share_vol': 415, 'trade_value': 99550.2, 'trade_cost': 250, 'spread_cost': -985.62, 'holding_value': 98562.5, 'ledger_value': -99800.2, 'invested_pre_trade': 716658.76, 'invested_post_trade': 815221.26, 'available_pre_trade': 236307.06, 'available_post_trade': 136506.86}
LEDGER ENTRY -> 


LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3723601413016437, 'ticker': 'GAW', 'trade_date': Timestamp('2017-05-29 00:00:00'), 'spread': 0.01, 'price': 1107.0, 'ask_price': 1118.07, 'bid_price': 1095.93, 'share_vol': 106, 'trade_value': 116168.58, 'trade_cost': 250, 'spread_cost': 1173, 'holding_value': 117342.0, 'ledger_value': 115918.58, 'invested_pre_trade': 860823.3099999999, 'invested_post_trade': 743481.3099999999, 'available_pre_trade': 69624.77000000002, 'available_post_trade': 185543.35000000003}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3633358681469089, 'ticker': 'SPT', 'trade_date': Timestamp('2017-05-29 00:00:00'), 'spread': 0.01, 'price': 121.0, 'ask_price': 122.21, 'bid_price': 119.79, 'share_vol': 816, 'trade_value': 99723.36, 'trade_cost': 250, 'spread_cost': -987.36, 'holding_value': 98736.0, 'ledger_value': -99973.36, 'invested_pre_trade': 743481.3099999999, 'invested_post_trade': 842217.3099999999, 'available_pre_trade': 185543.350000000

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3682635322435509, 'ticker': 'TSCO', 'trade_date': Timestamp('2017-12-04 00:00:00'), 'spread': 0.01, 'price': 204.55, 'ask_price': 206.6, 'bid_price': 202.5, 'share_vol': 528, 'trade_value': 106920.0, 'trade_cost': 250, 'spread_cost': 1080, 'holding_value': 108002.40000000001, 'ledger_value': 106670.0, 'invested_pre_trade': 851312.8099999998, 'invested_post_trade': 743310.4099999998, 'available_pre_trade': 57037.07000000001, 'available_post_trade': 163707.07}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3633358681469089, 'ticker': 'MAB', 'trade_date': Timestamp('2017-12-04 00:00:00'), 'spread': 0.01, 'price': 258.0, 'ask_price': 260.58, 'bid_price': 255.42, 'share_vol': 382, 'trade_value': 99541.56, 'trade_cost': 250, 'spread_cost': -985.56, 'holding_value': 98556.0, 'ledger_value': -99791.56, 'invested_pre_trade': 743310.4099999998, 'invested_post_trade': 841866.4099999998, 'available_pre_trade': 163707.07, 'availabl

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3497903172687425, 'ticker': 'FGP', 'trade_date': Timestamp('2018-04-09 00:00:00'), 'spread': 0.01, 'price': 112.4, 'ask_price': 113.52, 'bid_price': 111.28, 'share_vol': 718, 'trade_value': 79899.04, 'trade_cost': 250, 'spread_cost': 807, 'holding_value': 80703.2, 'ledger_value': 79649.04, 'invested_pre_trade': 790006.5899999999, 'invested_post_trade': 709303.3899999999, 'available_pre_trade': 90535.01000000001, 'available_post_trade': 170184.05}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3491206697941979, 'ticker': 'MARS', 'trade_date': Timestamp('2018-04-09 00:00:00'), 'spread': 0.01, 'price': 104.8, 'ask_price': 105.85, 'bid_price': 103.75, 'share_vol': 942, 'trade_value': 99710.7, 'trade_cost': 250, 'spread_cost': -987.22, 'holding_value': 98721.59999999999, 'ledger_value': -99960.7, 'invested_pre_trade': 709303.3899999999, 'invested_post_trade': 808024.9899999999, 'available_pre_trade': 170184.05, 'available_p

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3600502771457811, 'ticker': 'EIG', 'trade_date': Timestamp('2018-07-09 00:00:00'), 'spread': 0.01, 'price': 149.8, 'ask_price': 151.3, 'bid_price': 148.3, 'share_vol': 722, 'trade_value': 107072.6, 'trade_cost': 250, 'spread_cost': 1082, 'holding_value': 108155.6, 'ledger_value': 106822.6, 'invested_pre_trade': 766359.49, 'invested_post_trade': 658203.89, 'available_pre_trade': 87256.01999999992, 'available_post_trade': 194078.61999999994}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3564009357716494, 'ticker': 'MAB', 'trade_date': Timestamp('2018-07-09 00:00:00'), 'spread': 0.01, 'price': 261.8, 'ask_price': 264.42, 'bid_price': 259.18, 'share_vol': 377, 'trade_value': 99686.34, 'trade_cost': 250, 'spread_cost': -986.99, 'holding_value': 98698.6, 'ledger_value': -99936.34, 'invested_pre_trade': 658203.89, 'invested_post_trade': 756902.49, 'available_pre_trade': 194078.61999999994, 'available_post_trade': 94142.27999

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3496861844539072, 'ticker': 'ACA', 'trade_date': Timestamp('2018-11-19 00:00:00'), 'spread': 0.01, 'price': 190.0, 'ask_price': 191.9, 'bid_price': 188.1, 'share_vol': 641, 'trade_value': 120572.1, 'trade_cost': 250, 'spread_cost': 1218, 'holding_value': 121790.0, 'ledger_value': 120322.1, 'invested_pre_trade': 806076.8899999999, 'invested_post_trade': 684286.8899999999, 'available_pre_trade': 23449.33999999994, 'available_post_trade': 143771.43999999994}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3491206697941979, 'ticker': 'TALK', 'trade_date': Timestamp('2018-11-19 00:00:00'), 'spread': 0.01, 'price': 115.4, 'ask_price': 116.55, 'bid_price': 114.25, 'share_vol': 855, 'trade_value': 99650.25, 'trade_cost': 250, 'spread_cost': -986.67, 'holding_value': 98667.0, 'ledger_value': -99900.25, 'invested_pre_trade': 684286.8899999999, 'invested_post_trade': 782953.8899999999, 'available_pre_trade': 143771.43999999994, 'a

LEDGER ENTRY -> 
	{'trade_type': 'sell', 'signal_prob': 0.3600502771457811, 'ticker': 'AGR', 'trade_date': Timestamp('2019-01-28 00:00:00'), 'spread': 0.01, 'price': 58.5, 'ask_price': 59.09, 'bid_price': 57.91, 'share_vol': 1723, 'trade_value': 99778.93, 'trade_cost': 250, 'spread_cost': 1008, 'holding_value': 100795.5, 'ledger_value': 99528.93, 'invested_pre_trade': 787929.2799999999, 'invested_post_trade': 687133.7799999999, 'available_pre_trade': 16677.32999999993, 'available_post_trade': 116206.25999999992}
LEDGER ENTRY -> 
	{'trade_type': 'buy', 'signal_prob': 0.3491206697941979, 'ticker': 'MDC', 'trade_date': Timestamp('2019-01-28 00:00:00'), 'spread': 0.01, 'price': 313.2, 'ask_price': 316.33, 'bid_price': 310.07, 'share_vol': 315, 'trade_value': 99643.95, 'trade_cost': 250, 'spread_cost': -986.58, 'holding_value': 98658.0, 'ledger_value': -99893.95, 'invested_pre_trade': 687133.7799999999, 'invested_post_trade': 785791.7799999999, 'available_pre_trade': 116206.25999999992, 'av

In [16]:
#Show summary
print('rf_fund.st_val:£{:,.2f}'.format(rf_fund.st_val/100))
print('rf_fund.available:£{:,.2f}'.format(rf_fund.available/100))
print('rf_fund.codb:£{:,.2f}'.format(rf_fund.codb/100))
print('rf_fund.invested_value:£{:,.2f}'.format(rf_fund.invested_value/100))
rf_ledger_df = pd.DataFrame(rf_fund.ledger,columns=[
            'trade_type'
            ,'signal_prob'
            ,'ticker'
            ,'trade_date'
            ,'spread'
            ,'price'
            ,'ask_price'
            ,'bid_price'
            ,'share_vol'
            ,'trade_value'
            ,'trade_cost'
            ,'spread_cost'
            ,'holding_value'
            ,'ledger_value'
            ,'invested_pre_trade'
            ,'invested_post_trade'
            ,'available_pre_trade'
            ,'available_post_trade'])
print('TRADE COUNT:{:,}'.format(len(rf_ledger_df)))
rf_ledger_df.head(50)

rf_fund.st_val:£10,000.00
rf_fund.available:£360.68
rf_fund.codb:£342.90
rf_fund.invested_value:£7,461.26
TRADE COUNT:167


,trade_type,signal_prob,ticker,trade_date,spread,price,ask_price,bid_price,share_vol,trade_value,trade_cost,spread_cost,holding_value,ledger_value,invested_pre_trade,invested_post_trade,available_pre_trade,available_post_trade
0,buy,0.363336,AGR,2014-01-06,0.01,40.00,40.40,39.60,2469,99747.60,250,-987.60,98760.00,-99997.60,0.00,98760.00,1000000.00,900002.40
1,buy,0.363336,CNE,2014-01-06,0.01,271.40,274.11,268.69,363,99501.93,250,-985.18,98518.20,-99751.93,98760.00,197278.20,900002.40,800250.47
2,buy,0.363336,COA,2014-01-06,0.01,30.50,30.80,30.20,3238,99730.40,250,-987.59,98759.00,-99980.40,197278.20,296037.20,800250.47,700270.07
3,buy,0.363336,FGP,2014-01-06,0.01,137.40,138.77,136.03,718,99636.86,250,-986.53,98653.20,-99886.86,296037.20,394690.40,700270.07,600383.21
4,buy,0.363336,FUTR,2014-01-06,0.01,226.50,228.77,224.23,436,99743.72,250,-987.54,98754.00,-99993.72,394690.40,493444.40,600383.21,500389.49
5,buy,0.363336,GRG,2014-01-06,0.01,491.25,496.16,486.34,201,99728.16,250,-987.41,98741.25,-99978.16,493444.40,592185.65,500389.49,400411.33
6,buy,0.363336,LLOY,2014-01-06,0.01,83.29,84.12,82.46,1185,99682.20,250,-986.99,98698.65,-99932.20,592185.65,690884.30,400411.33,300479.13
7,buy,0.363336,RBS,2014-01-06,0.01,361.50,365.12,357.88,273,99677.76,250,-986.89,98689.50,-99927.76,690884.30,789573.80,300479.13,200551.37
8,buy,0.363336,SGRO,2014-01-06,0.01,325.05,328.30,321.80,303,99474.90,250,-984.90,98490.15,-99724.90,789573.80,888063.95,200551.37,100826.47
9,buy,0.363336,SHI,2014-01-06,0.01,213.00,215.13,210.87,463,99605.19,250,-986.19,98619.00,-99855.19,888063.95,986682.95,100826.47,971.28


In [17]:
#Get the completed trades
rf_trades_df = completed_trades(rf_ledger_df,rf_fund.st_val,rf_fund.cur_holdings)
rf_trades_df

TRADE COUNT -> 74
TOTAL PROFIT/LOSS -> £9,334.21

OPEN TRADES -> 19
OPEN TRADES VALUE -> £18,738.74

TOTAL ROI VALUE -> £28,072.95
TOTAL ROI % -> 180.73%


,ticker,share_vol,buy_spend,buy_prob,buy_date,sell_spend,sell_prob,sell_date,periods_held,profit_loss,holding_value
0,AGR,2469,99997.60,0.363336,2014-01-06,147618.41,0.372360,2015-03-30,64.0,47620.81,149374.50
1,AGR,1735,99960.45,0.363336,2017-01-09,101091.35,0.372464,2017-03-13,9.0,1130.90,102365.00
2,AGR,1723,99960.01,0.349121,2018-07-23,99528.93,0.360050,2019-01-28,27.0,-431.08,100795.50
3,AGR,1679,99965.81,0.360538,2019-02-25,98659.89,0.360316,2019-04-29,9.0,-1305.92,99900.50
4,CNE,363,99751.93,0.363336,2014-01-06,NaN,NaN,NaT,NaN,NaN,98518.20
5,COA,3238,99980.40,0.363336,2014-01-06,103916.46,0.372360,2014-02-03,4.0,3936.06,105235.00
6,COA,3440,99975.60,0.359199,2015-05-11,100198.00,0.349686,2015-10-05,21.0,222.40,101480.00
7,FGP,718,99886.86,0.363336,2014-01-06,79649.04,0.349790,2018-04-09,222.0,-20237.82,80703.20
8,FGP,858,99906.70,0.352443,2018-04-16,100324.76,0.349790,2018-05-14,4.0,418.06,101587.20
9,FGP,858,99992.50,0.352443,2018-05-21,NaN,NaN,NaT,NaN,NaN,98755.80


In [18]:
#Export
rf_ledger_df.to_csv(path+r'\trades_ledger_rf_mod.csv')